# Pydantic

🐍 **Pydantic** es una biblioteca para crear modelos de datos autovalidantes en Python, lo que facilita la definición y aplicación de estructuras de datos con **tipificación estricta**.

📋 Pydantic proporciona validación de datos, lo que garantiza que los modelos de datos cumplan con los tipos especificados y genera excepciones de validación si hay discrepancias de tipos.

Sus puntos fuertes son:

- **Sencillez**: Se basa en las “sugerencias de tipo” ya existentes en Python pero que sólo funcionan de modo “informativo”. Eso significa menos que aprender, menos código nuevo que escribir e integración con los IDE y herramientas de análisis estático.
- **Velocidad**: la lógica de validación central de Pydantic está escrita en Rust, como resultado, Pydantic es una de las bibliotecas de validación de datos más rápidas para Python.
- **Esquema JSON**: los modelos de Pydantic pueden emitir esquemas JSON, lo que permite una fácil integración con otras herramientas.
- **Modo estricto y laxo**: Pydantic puede ejecutarse en modo `strict=True` (donde los datos no se convierten) o en modo `strict=False`, donde Pydantic intenta convertir los datos al tipo correcto
cuando sea apropiado.
- **Dataclasses, TypedDicts y más**: Pydantic admite la validación de muchos tipos procedentes de bibliotecas estándar, incluidos `dataclass` y `TypedDict`.
- **Personalización**: Pydantic permite que los validadores y serializadores personalizados alteren la forma en que se procesan los datos.
- **Ecosistema**: alrededor de 8000 paquetes en **PyPI** usan Pydantic, incluidas bibliotecas muy populares como [**FastAPI**](https://github.com/tiangolo/fastapi), **[huggingface/transformers](https://github.com/huggingface/transformers)**, [**Django Ninja**](https://github.com/vitalik/django-ninja), [**SQLModel](https://github.com/tiangolo/sqlmodel)** y [**LangChain**](https://github.com/hwchase17/langchain).
- **Probado en campo de batalla**: Pydantic se descarga más de 70 millones de veces al mes y lo usan todas las empresas FAANG y 20 de las 25 empresas más grandes del NASDAQ. Si estás intentando hacer algo con Pydantic, es probable que alguien más ya lo haya hecho.

[Welcome to Pydantic - Pydantic](https://docs.pydantic.dev/2.6/)

## Instalación

La instalación de pydantic se realiza a través del administrador de paquetes:




In [ ]:
# !pip install pydantic



> ⚠️ Nota para Google Colab.
Actualmente Google Colab tiene instalado por defecto pydantic en su versión 1.10.
Este tutorial está hecho para la versión 2.
Por tanto hay que desinstalar pydantic e instalar la versión 2 de manera estricta.<br>
`!pip install fastapi kaleido python-multipart uvicorn cohere openai tiktoken`<br>
`!pip install "tensorflow-probability==0.23.0"`<br>
`!pip uninstall pydantic==1.10.14`<br>
`!pip install "pydantic==2.*"`<br>
>

Comprobamos la versión de pydantic instalada.




In [2]:
import pydantic
pydantic.__version__
# [Out: ] 2.9.2

'2.9.2'



## **Base Model**

Los modelos de datos de Pydantic son simplemente clases de Python con funcionalidad adicional proporcionada por Pydantic mediante herencia. Necesitamos importar el modelo base.




In [6]:
from pydantic import BaseModel



Este es un modelo Pydantic muy básico, como puede ver, definimos el tipo de datos de los campos en el modelo usando "sugerencias de tipo" de Python y lo añadimos a lo que heredamos del modelo base.




In [7]:
# heredamos "BaseModel" de pydantic

class Persona(BaseModel):
    nombre: str
    apellido1: str
    edad: int



Ahora podemos crear instancias de este modelo.




In [8]:
p = Persona(nombre="Juan", apellido1="Pérez", edad=25)
print(p)

nombre='Juan' apellido1='Pérez' edad=25




Si no usamos los tipos de datos que se espera para cada campo, obtenemos un mensaje de error "ValidationError".




In [11]:
error = Persona(nombre="Juan", apellido1=25, edad="Perez")

ValidationError: 2 validation errors for Persona
apellido1
  Input should be a valid string [type=string_type, input_value=25, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
edad
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Perez', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing



Los datos no confiables se pueden pasar a un modelo y, después del análisis y la validación, **Pydantic** garantiza que los campos de la instancia del modelo resultante se ajustarán a los tipos de campo definidos en el modelo. **Pydantic** es principalmente una biblioteca de análisis y transformación, no una biblioteca de validación.

La validación es un medio para un fin: crear un modelo que se ajuste a los tipos y restricciones proporcionados. En otras palabras, **Pydantic** garantiza los tipos y las restricciones del modelo de salida, no los datos de entrada. Aunque la validación no es el objetivo principal de **Pydantic**, puede utilizar esta biblioteca para una validación personalizada.

✅ **Pydantic**, en algunos casos, intentará convertir los datos de entrada al tipo adecuado, pero cuando no pueda hacerlo y cuando la validación falla, Pydantic genera un error de validación.

De esta manera, una vez que tengamos una instancia del modelo, tendremos la garantía de que el nombre será una cadena, el apellido será una cadena y la edad será un número entero. De hecho, también tenemos la garantía, debido a la forma en que definimos los campos, de que esos campos contendrán datos y no serán `None` porque, de forma predeterminada, todos los campos son obligatorios. *Veremos cómo crear campos opcionales más adelante.*

Podemos capturar el error para presentarlo en pantalla de manera más limpia




In [12]:
from pydantic import ValidationError

try:
    error = Persona(nombre="Juan", apellido1=25, edad="Perez")
except ValidationError as texto_error:
    print(texto_error)

2 validation errors for Persona
apellido1
  Input should be a valid string [type=string_type, input_value=25, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
edad
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Perez', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing




Podemos acceder a los campos de la instancia creada como en cualquier otra clase de Python




In [13]:
p = Persona(nombre="Juan", apellido1="Pérez", edad=25)

p.nombre, p.apellido1, p.edad

('Juan', 'Pérez', 25)



Incluso modificarlos.




In [14]:
p.edad = 26
p

Persona(nombre='Juan', apellido1='Pérez', edad=26)



Por defecto, Pydantic no valida los datos en estas modificaciones. Sólo en la creación de la instancia.




In [15]:
p.edad = "Veintiseis"
p

Persona(nombre='Juan', apellido1='Pérez', edad='Veintiseis')

## Datos de los errores de validación

Podemos obtener los datos que han producido el error de validación como un objeto. Esto es útil para tratar de darles una solución:




In [16]:
from pydantic import ValidationError
try:
    error = Persona(nombre="Juan", apellido1=25, edad="Perez")
except ValidationError as texto_error:
    objeto_error_validacion = texto_error



En formato lista de diccionarios




In [ ]:
objeto_error_validacion.errors()

[{'type': 'string_type',
  'loc': ('apellido1',),
  'msg': 'Input should be a valid string',
  'input': 25,
  'url': 'https://errors.pydantic.dev/2.9/v/string_type'},
 {'type': 'int_parsing',
  'loc': ('edad',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': 'Perez',
  'url': 'https://errors.pydantic.dev/2.9/v/int_parsing'}]



En formato string de json:




In [ ]:
objeto_error_validacion.json()

'[{"type":"string_type","loc":["apellido1"],"msg":"Input should be a valid string","input":25,"url":"https://errors.pydantic.dev/2.9/v/string_type"},{"type":"int_parsing","loc":["edad"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"Perez","url":"https://errors.pydantic.dev/2.9/v/int_parsing"}]'



- Manejo de errores
    
    Pydantic generará un error `ValidationError` cada vez que encuentre un error en los datos que está validando. Se generará una excepción independientemente de la cantidad de errores encontrados; ese error `ValidationError` contendrá información sobre todos los errores y cómo ocurrieron.
    
    


In [17]:
from pydantic import BaseModel, ValidationError

class Modelo(BaseModel):
    lista_de_ints: list[int]
    floatante: float

datos = dict(
    lista_de_ints=['1', 2, 'Hola'],
    floatante='esto no es un float',
    )

try:
    Modelo(**datos)
except ValidationError as e:
    print(e)


2 validation errors for Modelo
lista_de_ints.2
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Hola', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
floatante
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='esto no es un float', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/float_parsing


Es posible saltarnos el paso de validación. Lo haremos cuando el paso de validación sea costoso computacionalmente y ya hayamos validado los datos en otro momento,

In [18]:
from pydantic import BaseModel
class User(BaseModel):
    id: int
    age: int
    name: str = 'John Doe'
original_user = User(id=123, age=32)
user_data = original_user.model_dump()
print(user_data)
#> {'id': 123, 'age': 32, 'name': 'John Doe'}
fields_set = original_user.model_fields_set
print(fields_set)
#> {'age', 'id'}
# ...
# pass user_data and fields_set to RPC or save to the database etc.
# ...
# you can then create a new instance of User without
# re-running validation which would be unnecessary at this point:
# if _fields_set was not provided, new_user.model_fields_set would be {'id', 'age', 'name'}.
new_user = User.model_construct(_fields_set=fields_set, **user_data)
print(repr(new_user))#> User(id=123, age=32, name='John Doe')
print(new_user.model_fields_set)
#> {'age', 'id'}
# construct can be dangerous, only use it with validated data!:
bad_user = User.model_construct(id='dog')
print(repr(bad_user))
#> User(id='dog', name='John Doe')

{'id': 123, 'age': 32, 'name': 'John Doe'}
{'age', 'id'}
User(id=123, age=32, name='John Doe')
{'age', 'id'}
User(id='dog', name='John Doe')



    

## Deserialización de los datos

Puedo convertir en instancia unos datos recibidos en formato diccionario




In [30]:
datos_dict = {"nombre": "Luis", "apellido1": "López", "edad": 30}
p = Persona.model_validate(datos_dict)
print(p)

nombre='Luis' apellido1='López' edad=30




o un string con formato json




In [20]:
datos_json = """{
    "nombre": "Luis",
    "apellido1": "López",
    "edad": 30}
    """
p = Persona.model_validate_json(datos_json)
print(p)

nombre='Luis' apellido1='López' edad=30




## Serialización de datos

La serialización es la operación inversa. Pasar de la instancia de la clase Persona a los tipos estándar (diccionario o string de json)




In [26]:
p.model_dump() # dump significa verteer

{'nombre': 'Luis', 'apellido1': 'López', 'edad': 30}

In [27]:
p.model_dump()["nombre"] # dump significa verteer

'Luis'

In [28]:
p.model_dump_json()

'{"nombre":"Luis","apellido1":"López","edad":30}'



## Campos *Required* y *Optional*

Como hemos dicho antes, el comportamiento por defecto es que todos los campos sean requeridos (obligatorios). Pero podemos modificar la forma en la que declaramos algunos campos, dándoles un contenido por defecto para que sean opcionales.

Por ejemplo, voy a añadir el campo apellido2 que contenga el segundo apellido de la persona pero, como algunos extranjeros no tienen segundo apellido voy a permitir que ese valor esté vacío.




In [31]:
class Persona(BaseModel):
    nombre: str
    apellido1: str
    apellido2: str = "" # Por si queremos crear objeto sin uno de los atributos
    edad: int

In [32]:
datos_dict = {"nombre": "Luis", "apellido1": "López", "apellido2": "García", "edad": 30}
p = Persona.model_validate(datos_dict)
print(p)

nombre='Luis' apellido1='López' apellido2='García' edad=30


In [33]:
datos_dict = {"nombre": "John", "apellido1": "Smith", "edad": 30}
p2 = Persona.model_validate(datos_dict)
print(p2)

nombre='John' apellido1='Smith' apellido2='' edad=30




Podemos ver cómo se han definido los campos del modelo con el atributo `model_fields`




In [34]:
Persona.model_fields

{'nombre': FieldInfo(annotation=str, required=True),
 'apellido1': FieldInfo(annotation=str, required=True),
 'apellido2': FieldInfo(annotation=str, required=False, default=''),
 'edad': FieldInfo(annotation=int, required=True)}



## Campos Nullable

En lugar de una cadena vacía, podríamos preferir guardar el valor `None` en apellido2 en caso de no haber segundo apellido. Pero hemos de tener cuidado de que el valor que asignamos por defecto (a la derecha del igual) sea del mismo tipo que el tipo que hemos asignado al campo. Y `None` no es de tipo `str` así que tengo que incluir el tipo `None` en la definición del campo:




In [ ]:
class Persona(BaseModel):
    nombre: str
    apellido1: str
    apellido2: str | None = None
    edad: int

In [ ]:
Persona.model_fields

{'nombre': FieldInfo(annotation=str, required=True),
 'apellido1': FieldInfo(annotation=str, required=True),
 'apellido2': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'edad': FieldInfo(annotation=int, required=True)}

In [35]:
datos_dict = {"nombre": "John", "apellido1": "Smith", "edad": 30}
p2 = Persona.model_validate(datos_dict)
print(p2)

nombre='John' apellido1='Smith' apellido2='' edad=30


Con la clase `Optional` de la librería `typing` podemos obtener lo mismo:

In [36]:
from typing import Optional
class Persona(BaseModel):
    nombre: str
    apellido1: str
    apellido2: Optional[str] = None
    edad: int
Persona.model_fields

{'nombre': FieldInfo(annotation=str, required=True),
 'apellido1': FieldInfo(annotation=str, required=True),
 'apellido2': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'edad': FieldInfo(annotation=int, required=True)}



Pero, OJO, eso no quiere decir que la introducción de apellido2 sea opcional, sino que es opcional que tenga un string o un None, porque si quito el `= None` de después, cuando intente crear la instancia de John Smith tendre que decir explícitamente `apellido2 = None`.

## Alias y la clase *Field*

Podría ser que recibiéramos datos en formato json en los que las claves no tienen un formato correcto que sirva para una variable de Python. Por ejemplo, que contengan espacios.




In [38]:
datos_json = """{
    "nombre": "Luis",
    "apellido paterno": "López",
    "edad": 30}
    """
p = Persona.model_validate_json(datos_json)
print(p)

"""----------------------
Devuelve un error de validación"""

nombre='Luis' apellido1='López' apellido2=None edad=30


'----------------------\nDevuelve un error de validación'



En este caso, yo no puedo crear el campo `apellido paterno`, y si creo el campo `apellido_paterno`, no me lo relacionará con la clave del json al deserializar con `model_validate_json`.

La solución es que permita usar un alias para ese campo. La clase Field me permite crear campos más complejos.

[Fields - Pydantic](https://docs.pydantic.dev/latest/concepts/fields/)




In [37]:
from typing import Optional
from pydantic import Field

class Persona(BaseModel):
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    edad: int

Persona.model_fields

{'nombre': FieldInfo(annotation=str, required=True),
 'apellido1': FieldInfo(annotation=str, required=True, alias='apellido paterno', alias_priority=2),
 'apellido2': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='apellido materno', alias_priority=2),
 'edad': FieldInfo(annotation=int, required=True)}

In [39]:
datos_json = """{
    "nombre": "Luis",
    "apellido paterno": "López",
    "edad": 30}
    """
p = Persona.model_validate_json(datos_json)
print(p)

nombre='Luis' apellido1='López' apellido2=None edad=30




Al serializar a un diccionario nos devolverá los nombres de los campos en las claves.




In [41]:
p.model_dump()

{'nombre': 'Luis', 'apellido1': 'López', 'apellido2': None, 'edad': 30}



Si queremos que serialice con los aliases:




In [42]:
p.model_dump(by_alias=True)

{'nombre': 'Luis',
 'apellido paterno': 'López',
 'apellido materno': None,
 'edad': 30}



Lo mismo ocurrirá con `model_dump_json`

Pero no tenemos opción de deserializar con los nombres de los campos, sólo con los aliases.




In [43]:
datos_json = """{
"nombre": "Luis",
"apellido1": "López",
"edad": 30}
"""
p = Persona.model_validate_json(datos_json)
print(p)

"""------------------
ValidationError: 1 validation error for Persona
apellido paterno
  Field required [type=missing, input_value={'nombre': 'Luis', 'apell...': 'López', 'edad': 30}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
"""

ValidationError: 1 validation error for Persona
apellido paterno
  Field required [type=missing, input_value={'nombre': 'Luis', 'apell...': 'López', 'edad': 30}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing



para poderlo hacer necesitamos manipular la configuración del modelo.

## Configuración del modelo

La configuración del modelo se importa de pydantic y podemos sobreescribir el comportamiento por defecto `populate_by_name = False` que impide usar el nombre del campo para cargar los datos en la instancia.

[Configuration - Pydantic](https://docs.pydantic.dev/latest/concepts/config/)




In [46]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional
class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    edad: int
Persona.model_fields

{'nombre': FieldInfo(annotation=str, required=True),
 'apellido1': FieldInfo(annotation=str, required=True, alias='apellido paterno', alias_priority=2),
 'apellido2': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='apellido materno', alias_priority=2),
 'edad': FieldInfo(annotation=int, required=True)}

In [48]:
datos_json = """{
    "nombre": "Luis",
    "apellido1": "López",
    "apellido materno": "María",
    "edad": 30}
    """
p = Persona.model_validate_json(datos_json)
print(p)

nombre='Luis' apellido1='López' apellido2='María' edad=30


In [ ]:
class Libro

In [50]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional
class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    edad: int
Persona.model_fields

{'nombre': FieldInfo(annotation=str, required=True),
 'apellido1': FieldInfo(annotation=str, required=True, alias='apellido paterno', alias_priority=2),
 'apellido2': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='apellido materno', alias_priority=2),
 'edad': FieldInfo(annotation=int, required=True)}

In [60]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional

class Datos_Libro(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    titulo: str
    autor: str
    anio: Optional[int] = Field(alias="año", default = None)
    isbn: int = Field(alias="referencia isbn")
    paginas: int | None = Field(alias="número de páginas", default = None)
Datos_Libro.model_fields

{'titulo': FieldInfo(annotation=str, required=True),
 'autor': FieldInfo(annotation=str, required=True),
 'anio': FieldInfo(annotation=Union[int, NoneType], required=False, default=None, alias='año', alias_priority=2),
 'isbn': FieldInfo(annotation=int, required=True, alias='referencia isbn', alias_priority=2),
 'paginas': FieldInfo(annotation=Union[int, NoneType], required=False, default=None, alias='número de páginas', alias_priority=2)}

In [62]:
l1 = Datos_Libro(titulo = "Avatar", autor = "Whoever", anio = 2013, isbn =2345)
l1

Datos_Libro(titulo='Avatar', autor='Whoever', anio=2013, isbn=2345, paginas=None)

In [59]:
datos_json = """{
    "titulo": "Avatar",
    "autor": "Whoever",
    "anio": "2013",
    "referencia isbn": "2345",
    "número de páginas": 200
    }"""
p = Datos_Libro.model_validate_json(datos_json)
print(p)

titulo='Avatar' autor='Whoever' anio=2013 isbn=2345 paginas=200




## Valores por defecto dinámicos (Default Factories)

Los "Default Factories" (fábricas por defecto) son funciones que se utilizan para generar valores por defecto dinámicamente cuando se inicializa un modelo de datos. Estas funciones se definen en el cuerpo de una clase modelo de Pydantic y se asocian con un campo específico mediante el uso de `Field`.

La idea principal detrás de las "Default Factories" es proporcionar una manera flexible y dinámica de establecer valores predeterminados para los campos de un modelo Pydantic. Esto es útil cuando se desea calcular un valor por defecto en tiempo de ejecución en lugar de simplemente asignar un valor estático.

Para ello hay que especificar funciones que serán invocadas automáticamente para generar un valor por defecto para un campo específico. Estas fábricas se definen como métodos dentro de la clase del modelo y se decoran con `@Field(default_factory=...)`. Cuando se crea una instancia del modelo y no se proporciona un valor para ese campo en particular, la fábrica por defecto se ejecuta para calcular el valor.




In [66]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List

def calcular_valor_predeterminado():
    # Esta función puede realizar cálculos u obtener valores dinámicamente.
    return "¡Valor Dinámico!"

class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    campo_con_default_factory: str = Field(default_factory=calcular_valor_predeterminado)
    lista_con_default_factory: List[int] = Field(default_factory=list)
    edad: int



print(f"Campos del modelo: Persona.model_fields")

# Crear una instancia del modelo sin proporcionar valores específicos para los campos con default factory
p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad=50)

# Acceder a los valores, incluyendo los generados por las fábricas por defecto
print(p.model_dump())
print(p.model_dump_json())

Campos del modelo: Persona.model_fields
{'nombre': 'Juan', 'apellido1': 'Pérez', 'apellido2': 'García', 'campo_con_default_factory': '¡Valor Dinámico!', 'lista_con_default_factory': [], 'edad': 50}
{"nombre":"Juan","apellido1":"Pérez","apellido2":"García","campo_con_default_factory":"¡Valor Dinámico!","lista_con_default_factory":[],"edad":50}




En este ejemplo, `campo_con_default_factory` utilizará la función `calcular_valor_predeterminado` como su fábrica por defecto. Cuando se crea una instancia de `MiModelo` y no se proporciona un valor para `campo_con_default_factory`, la función se ejecutará automáticamente para obtener el valor predeterminado.

La fábrica por defecto también se puede utilizar para campos de tipo lista (`lista_con_default_factory` en este caso), donde proporcionará una lista vacía como valor predeterminado.

Vamos a crear un caso práctico de uso de `default_factory`. Podemos hacer que ese campo generado guarde el momento en el que se ha creado la instancia:




In [69]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from datetime import datetime, timezone


class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    dt: datetime = Field(default_factory=Persona.obtener_fecha_hora())
    lista_con_default_factory: List[int] = Field(default_factory=list)
    edad: int
    @staticmethod
    def obtener_fecha_hora():
        # Esta función puede realizar cálculos u obtener valores dinámicamente.
        return datetime.now(timezone.utc)

Persona.model_fields
# Crear una instancia del modelo sin proporcionar valores específicos para los campos con default factory
p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad=50)
# Acceder a los valores, incluyendo los generados por las fábricas por defecto
print(p.model_dump())
print(p.model_dump_json())


{'nombre': 'Juan', 'apellido1': 'Pérez', 'apellido2': 'García', 'dt': datetime.datetime(2024, 10, 9, 9, 10, 53, 969639, tzinfo=datetime.timezone.utc), 'lista_con_default_factory': [], 'edad': 50}
{"nombre":"Juan","apellido1":"Pérez","apellido2":"García","dt":"2024-10-09T09:10:53.969639Z","lista_con_default_factory":[],"edad":50}




Podemos ver que la serialización del dato es distinta si lo hacemos a un diccionario de Python o a un string json. En el primer caso, dt almacena un objeto de tipo datetime, con todos sus atributos que serían accesibles usando Python. En el segundo, convierte la información a un string.

Si queremos un comportamiento distinto del comportamiento por defecto debemos sobreescribir los métodos de serialización mediante serializadores customizados.

## Serializadores customizados - Custom Serializers

Imaginemos que queremos guardar la altura de una persona en metros y al serializar queremos que nos devuelva el número con dos decimales.

Los serializadores son decoradores que se importan de pydantic y a los cuales hay que especificarles el campo que van a modificar en el proceso de serializado.

Como son métodos de la clase, recibirán el objeto self en los parámetros, además del valor a modificar




In [ ]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from pydantic import field_serializer
class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    edad: int
    altura: float
    @field_serializer("altura")
    def serialize_float(self, value):
        return round(value, 2)
# Crear una instancia del modelo sin proporcionar valores específicos para los campos con default factory
p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad=50, altura=1.684)
# Acceder a los valores, incluyendo los generados por las fábricas por defecto
print(p.model_dump())
print(p.model_dump_json())
-------------------------------------
{'nombre': 'Juan', 'apellido1': 'Pérez', 'apellido2': 'García', 'edad': 50, 'altura': 1.68}
{"nombre":"Juan","apellido1":"Pérez","apellido2":"García","edad":50,"altura":1.68}



Vamos a customizar ahora el campo dt para que me devuelva la fecha y la hora en el formato que yo quiera cuando convierto el dato en un json




In [ ]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from pydantic import field_serializer
from datetime import datetime, timezone
def obtener_fecha_hora():
    # Esta función puede realizar cálculos u obtener valores dinámicamente.
    return datetime.now(timezone.utc)
class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    dt: datetime = Field(default_factory=obtener_fecha_hora)
    edad: int
    altura: float
    @field_serializer("altura")
    def serialize_float(self, value):
        return round(value, 2)

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_float(self, value):
        return value.strftime("%Y/%m/%d %I:%M %p")
# Crear una instancia del modelo sin proporcionar valores específicos para los campos con default factory
p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad=50, altura=1.684)
# Acceder a los valores, incluyendo los generados por las fábricas por defecto
print(p.model_dump())
print(p.model_dump_json())
-------------------------------------
{'nombre': 'Juan', 'apellido1': 'Pérez', 'apellido2': 'García', 'dt': datetime.datetime(2024, 2, 5, 11, 25, 49, 623638, tzinfo=datetime.timezone.utc), 'edad': 50, 'altura': 1.684}
{"nombre":"Juan","apellido1":"Pérez","apellido2":"García","dt":"2024/02/05 11:25 AM","edad":50,"altura":1.684}
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_model_construction.py:53: UserWarning: `serialize_float` overrides an existing Pydantic `@field_serializer` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')



## Validadores Customizados - Custom Validators

Podemos introducir cierto grado de customización en la validación con Field (ver la línea `edad: int = Field(ge=0, le=120)` que limita la edad a enteros positivos y menores de 120)




In [ ]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from pydantic import field_serializer
from datetime import datetime, timezone

def obtener_fecha_hora():
    # Esta función puede realizar cálculos u obtener valores dinámicamente.
    return datetime.now(timezone.utc)

class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    dt: datetime = Field(default_factory=obtener_fecha_hora)
    edad: int = Field(ge=0, le=120)
    altura: float

    @field_serializer("altura")
    def serialize_float(self, value):
        return round(value, 2)

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_float(self, value):
        return value.strftime("%Y/%m/%d %I:%M %p")

p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad=50, altura=1.684)
print(p.model_dump())
print(p.model_dump_json())




Pero los validadores customizados nos permiten transformar el dato recibido. Por ejemplo. Si se introduce un negativo se guarda el valor absoluto.

Los validadores son métodos de la clase, no de la instancia. Por ello requieren el decorador "classmethod" y la función decorada requiere la clase como primer parámetro.




In [71]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from pydantic import field_serializer, field_validator
from datetime import datetime, timezone
def obtener_fecha_hora():
    # Esta función puede realizar cálculos u obtener valores dinámicamente.
    return datetime.now(timezone.utc)
class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default = None)
    dt: datetime = Field(default_factory=obtener_fecha_hora)
    edad: int = Field(ge=-120, le=120)
    altura: float
    @field_serializer("altura")
    def serialize_float(self, value):
        return round(value, 2)

    # Los validadores son métodos de la clase, no de la instancia
    @field_validator("edad")
    @classmethod
    def deserializa_edad(cls, value):
        return abs(value)

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_float(self, value):
        return value.strftime("%Y/%m/%d %I:%M %p")

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_model_construction.py:66: UserWarning: `serialize_float` overrides an existing Pydantic `@field_serializer` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')


In [72]:
p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad=-50, altura=1.684)
print(p.model_dump())
print(p.model_dump_json())

{'nombre': 'Juan', 'apellido1': 'Pérez', 'apellido2': 'García', 'dt': datetime.datetime(2024, 10, 9, 10, 8, 15, 806769, tzinfo=datetime.timezone.utc), 'edad': 50, 'altura': 1.684}
{"nombre":"Juan","apellido1":"Pérez","apellido2":"García","dt":"2024/10/09 10:08 AM","edad":50,"altura":1.684}


## Validador de DNIs

In [79]:
from pydantic import BaseModel, Field, ConfigDict, field_validator, field_serializer
from datetime import datetime, timezone

def obtener_fecha_hora():
    # Esta función puede realizar cálculos u obtener valores dinámicamente.
    return datetime.now(timezone.utc)

class Persona(BaseModel):
    model_config = ConfigDict(populate_by_name=True)
    nombre: str
    apellido1: str = Field(alias="apellido paterno")
    apellido2: str | None = Field(alias="apellido materno", default=None)
    dni: str = Field(min_length=9, max_length=9, alias="DNI")
    dt: datetime = Field(default_factory=obtener_fecha_hora)
    edad: int = Field(ge=-120, le=120)  # Rango de edad permitido entre -120 y 120
    altura: float

    # Validador para el DNI
    @field_validator("dni")
    @classmethod
    def validar_dni(cls, value):
        letras_dni = "TRWAGMYFPDXBNJZSQVHLCKE"
        if len(value) == 9 and value[:8].isdigit() and value[-1].isalpha():
            numero = int(value[:8])
            letra = value[-1].upper()
            if letra == letras_dni[numero % 23]:
                return value
        raise ValueError("DNI no válido")

    # Serializador para la altura
    @field_serializer("altura")
    def serialize_float(self, value):
        return round(value, 2)

    # Validador para deserializar la edad (convertir a valor positivo si es negativo)
    @field_validator("edad")
    @classmethod
    def deserializa_edad(cls, value):
        return abs(value)

    # Serializador para la fecha y hora en formato específico
    @field_serializer("dt", when_used="json-unless-none")
    def serialize_datetime(self, value):
        return value.strftime("%Y/%m/%d %I:%M %p")





In [84]:
p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad="-50", altura=1.684, dni="12345678Z")
print(p.model_dump())
print(p.model_dump_json())

{'nombre': 'Juan', 'apellido1': 'Pérez', 'apellido2': 'García', 'dni': '12345678Z', 'dt': datetime.datetime(2024, 10, 9, 10, 24, 40, 734544, tzinfo=datetime.timezone.utc), 'edad': 50, 'altura': 1.68}
{"nombre":"Juan","apellido1":"Pérez","apellido2":"García","dni":"12345678Z","dt":"2024/10/09 10:24 AM","edad":50,"altura":1.68}




Puedo introducir la edad como string y aún así no voy a tener problemas con `abs(value)` porque la validación de string a entero se realiza antes del field_validator. Es decir, field_validator es un validador "a posteriori"




In [ ]:
p = Persona(nombre="Juan", apellido1="Pérez", apellido2="García", edad="-50", altura=1.684)
print(p.model_dump())
print(p.model_dump_json())
---------------------------------------
{'nombre': 'Juan', 'apellido1': 'Pérez', 'apellido2': 'García', 'dt': datetime.datetime(2024, 2, 5, 11, 47, 50, 150111, tzinfo=datetime.timezone.utc), 'edad': 50, 'altura': 1.684}
{"nombre":"Juan","apellido1":"Pérez","apellido2":"García","dt":"2024/02/05 11:47 AM","edad":50,"altura":1.684}



### Validar lista de elementos. Queremos que sean únicos.




In [85]:
from pydantic import BaseModel
from pydantic import field_validator
class Model(BaseModel):
    numeros: list[int] = []
    @field_validator("numeros")
    @classmethod
    def asegura_unicos(csl, numeros):
        if len(set(numeros)) != len(numeros):
            raise ValueError("Los elementos han de ser únicos")
        return numeros

In [87]:
lista_numeros = Model(numeros = [1, 2, 3])
print(lista_numeros.numeros)


ValidationError: 1 validation error for Model
numeros
  Value error, Los elementos han de ser únicos [type=value_error, input_value=[1, 3, 3], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error



Si la lista de números se puede convertir en lista de enteros no da error




In [ ]:
lista_numeros = Model(numeros = [1, "2", 3.0])
print(lista_numeros.numeros)



Pero si introducimos un número repetido se lanza la excepción.




In [ ]:
from pydantic import ValidationError
try:
    lista_numeros = Model(numeros = [1, 1, "2", 3.0])
except ValidationError as ex:
    print(ex)
---------------------------
1 validation error for Model
numeros
  Value error, Los elementos han de ser únicos [type=value_error, input_value=[1, 1, '2', 3.0], input_type=list]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error



## Modelos de datos anidados

Supongamos que obtenemos los siguientes datos de algún lugar (en formato diccionario)




In [88]:
datos = {
    "nombre": "Pedro",
    "apellido1": "Gómez",
    "apellido2": "Pérez",
    "nacimiento": {
        "lugar": {
            "pais": "Colombia",
            "ciudad": "Medellín"},
        "fecha": "1990-01-15"
    }
}



Vemos datos anidados dentro de "nacimiento" y de "lugar”

Vamos definiendo los modelos de datos desde el anidamiento máximo hacia arriba. Así podemos usar el tipo de los datos anidados en los niveles superiores.




In [90]:
class Lugar(BaseModel):
    pais: str
    ciudad: str

In [91]:
from datetime import date
from pydantic import Field
class Nacimiento(BaseModel):
    lugar: Lugar
    dt: date = Field(alias="fecha")

In [92]:
class Persona(BaseModel):
    nombre: str
    apellido1: str
    apellido2: str | None = None
    nacimiento: Nacimiento

In [93]:
pedro = Persona.model_validate(datos)
pedro
# -----------------------------
# Persona(nombre='Pedro', apellido1='Gómez', apellido2='Pérez', nacimiento=Nacimiento(lugar=Lugar(pais='Colombia', ciudad='Medellín'), dt=datetime.date(1990, 1, 15)))

Persona(nombre='Pedro', apellido1='Gómez', apellido2='Pérez', nacimiento=Nacimiento(lugar=Lugar(pais='Colombia', ciudad='Medellín'), dt=datetime.date(1990, 1, 15)))

In [94]:
pedro.nacimiento

Nacimiento(lugar=Lugar(pais='Colombia', ciudad='Medellín'), dt=datetime.date(1990, 1, 15))

In [95]:
pedro.nacimiento.dt


datetime.date(1990, 1, 15)

In [97]:
pedro.nacimiento

Nacimiento(lugar=Lugar(pais='Colombia', ciudad='Medellín'), dt=datetime.date(1990, 1, 15))

In [96]:
pedro.nacimiento.lugar

Lugar(pais='Colombia', ciudad='Medellín')

In [ ]:
pedro.nacimiento.lugar.pais

In [ ]:
print(pedro.model_dump_json(indent=4))
--------------------------------
{
    "nombre": "Pedro",
    "apellido1": "Gómez",
    "apellido2": "Pérez",
    "nacimiento": {
        "lugar": {
            "pais": "Colombia",
            "ciudad": "Medellín"
        },
        "dt": "1990-01-15"
    }
}

In [ ]:
from pprint import pprint
print(pedro.model_dump())
--------------------------------
{'apellido1': 'Gómez',
 'apellido2': 'Pérez',
 'nacimiento': {'dt': datetime.date(1990, 1, 15),
                'lugar': {'ciudad': 'Medellín', 'pais': 'Colombia'}},
 'nombre': 'Pedro'}



- Recursive models
    
    More complex hierarchical data structures can be defined using models themselves as types in annotations.
    
    


In [ ]:
    from pydantic import BaseModel

    class Foo(BaseModel):
    	count: int
    	size: float | None = None

    class Bar(BaseModel):
    	apple: str = 'x'
    	banana: str = 'y'

    class Spam(BaseModel):
    	foo: Foo
    	bars: list[Bar]

    m = Spam(foo={'count': 4}, bars=[{'apple': 'x1'}, {'apple': 'x2'}])
    print(m)
    """
    foo=Foo(count=4, size=None)
    bars=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
    """
    print(m.model_dump())
    """
    {'foo': {'count': 4, 'size': None},
    'bars': [
    	{'apple': 'x1', 'banana': 'y'},
    	{'apple': 'x2', 'banana': 'y'}]}
    """



    
    Model instances will be parsed recursively as well as at the top level.
    Here a vanilla class is used to demonstrate the principle, but any class could be used instead.
    
    


In [ ]:
    from pydantic import BaseModel, ConfigDict

    class PetCls:
    	def **init**(self, *, name: str, species: str):
    		[self.name](http://self.name/) = name
    		self.species = species

    class PersonCls:
    	def **init**(self, *, name: str, age: float = None, pets: list[PetCls]):
    		[self.name](http://self.name/) = name
    		self.age = age
    		self.pets = pets

    class Pet(BaseModel):
    	model_config = ConfigDict(from_attributes=True)
    	name: str
    	species: str

    class Person(BaseModel):model_config = ConfigDict(from_attributes=True)
    	name: str
    	age: float = None
    	pets: list[Pet]

    bones = PetCls(name='Bones', species='dog')
    orion = PetCls(name='Orion', species='cat')
    anna = PersonCls(name='Anna', age=20, pets=[bones, orion])
    anna_model = Person.model_validate(anna)
    print(anna_model)
    """
    name='Anna'
    age=20.0
    pets=[Pet(name='Bones', species='dog'), Pet(name='Orion', species='cat')]
    """



    

# Ejercicios

### Ejercicio 1: Base Model

1. Crea un modelo Pydantic llamado `Product` con campos como `name` (str), `price` (float), y `is_available` (bool).



In [98]:
from pydantic import BaseModel

class Product(BaseModel):
    name: str
    price: float
    is_available: bool

In [99]:
product = Product(name="Camiseta", price=29.99, is_available=True)
print(product)

name='Camiseta' price=29.99 is_available=True


### Ejercicio 2: Datos de errores de validación

1. Intenta crear una instancia de `Product` sin proporcionar el precio y observa el error de validación.


In [100]:
product = Product(name="Camiseta", is_available=True)
print(product)

ValidationError: 1 validation error for Product
price
  Field required [type=missing, input_value={'name': 'Camiseta', 'is_available': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

2. Modifica el modelo `Product` para que el campo `price` sea opcional.



In [101]:
from pydantic import BaseModel
from typing import Optional

class Product(BaseModel):
    name: str
    price: Optional[float] = None
    is_available: bool

In [102]:
product = Product(name="Camiseta", is_available=True)
print(product)

name='Camiseta' price=None is_available=True


### Ejercicio 3: Deserialización y Serialización

1. Crea un diccionario con datos de un producto y deserialízalo usando el modelo `Product`.


In [105]:
from pydantic import BaseModel
from typing import Optional

class Product(BaseModel):
    name: str
    price: Optional[float] = None
    is_available: bool

datos = {
    "name": "Camiseta",
    "price": 29.99,
    "is_available": True
}

datos2 = '''{
    "name": "Camiseta",
    "price": 29.99,
    "is_available": True
}'''

product = Product.model_validate(datos)
product2 = Product.model_validate_json(datos2)
print(product)

name='Camiseta' price=29.99 is_available=True


2. Serializa la instancia de `Product` creada en el Ejercicio 1 y verifica que los datos serializados sean correctos.



In [ ]:
product.model_dump()

### Ejercicio 4: Campos Required, Optional y Nullables

1. Modifica el modelo `Product` para que el campo `is_available` sea opcional y nulo.


In [108]:
from pydantic import BaseModel, Field
from typing import Optional

class Product(BaseModel):
    name: str
    price: Optional[float] = None
    is_available: Optional[bool] = None

2. Intenta crear una instancia de `Product` sin proporcionar la disponibilidad y verifica que no genere un error.


In [109]:
datos = {
    "name": "Camiseta",
    "price": 29.99
}

product = Product.model_validate(datos)
print(product)

name='Camiseta' price=29.99 is_available=None


### Ejercicio 5: Alias y clase Field

1. Agrega un alias al campo `name` del modelo `Product` para que sea accesible como `product_name`.


In [111]:
from pydantic import BaseModel, Field
from typing import Optional

class Product(BaseModel):
    name: str
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)

datos = {
    "name": "Camiseta",
    "price": 29.99,
    "Esta disponible": True
}

product = Product.model_validate(datos)
print(product)


name='Camiseta' price=29.99 is_available=True


2. Deserializa un diccionario que contenga la clave `product_name` y serializa el objeto obtenido a un JSON



In [115]:
from pydantic import BaseModel, Field
from typing import Optional

class Product(BaseModel):
    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)

datos = {
    "product_name": "Camiseta",
    "price": 29.99,
    "is_available": True
}

product = Product.model_validate(datos)
print(product.model_dump())

{'name': 'Camiseta', 'price': 29.99, 'is_available': None}


### Ejercicio 6: Configuración del modelo

1. Configura el modelo `Product` para que ignore campos adicionales durante la validación.


In [ ]:
from pydantic import BaseModel, Field
from typing import Optional

class Product(BaseModel):
    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)
    searial_number: Optional[int] = None

datos = {
    "product_name": "Camiseta",
    "price": 29.99,
    "is_available": True
}

product = Product.model_validate(datos)
print(product.model_dump())

In [116]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional

class Product(BaseModel):
    model_config = ConfigDict(extra="ignore")  # Ignorar campos adicionales

    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)
    serial_number: Optional[int] = None

datos = {
    "product_name": "Camiseta",
    "price": 29.99,
    "is_available": True,
    "extra_field": "Este campo será ignorado"
}

product = Product.model_validate(datos)
print(product.model_dump())

{'name': 'Camiseta', 'price': 29.99, 'is_available': None, 'serial_number': None}


2. Intenta crear una instancia de `Product` con un campo adicional y verifica que no genere un error.



In [118]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional

class Product(BaseModel):
    model_config = ConfigDict(extra="allow")  # Ignorar campos adicionales

    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)
    serial_number: Optional[int] = None

datos = {
    "product_name": "Camiseta",
    "price": 29.99,
    "is_available": True,
    "extra_field": "Este campo será añadido"
}

product = Product.model_validate(datos)
print(product.model_dump())

{'name': 'Camiseta', 'price': 29.99, 'is_available': True, 'serial_number': None, 'extra_field': 'Este campo será añadido'}


In [119]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional

class Product(BaseModel):
    model_config = ConfigDict(extra="forbid")  # Ignorar campos adicionales

    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)
    serial_number: Optional[int] = None

datos = {
    "product_name": "Camiseta",
    "price": 29.99,
    "is_available": True,
    "extra_field": "Este campo será añadido"
}

product = Product.model_validate(datos)
print(product.model_dump())

ValidationError: 2 validation errors for Product
is_available
  Extra inputs are not permitted [type=extra_forbidden, input_value=True, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden
extra_field
  Extra inputs are not permitted [type=extra_forbidden, input_value='Este campo será añadido', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden

### Ejercicio 7: Valores por defecto dinámicos (Default Factories)

1. Agrega un campo `creation_date` al modelo `Product` que tenga como valor predeterminado la fecha actual.


In [130]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from datetime import datetime, timezone

class Product(BaseModel):
    model_config = ConfigDict(extra="allow")  # Ignorar campos adicionales

    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)
    serial_number: Optional[int] = None
    creation_date: str = Field(default_factory=lambda: Product.obtener_fecha_hora())

    @staticmethod
    def obtener_fecha_hora():
        # Esta función puede realizar cálculos u obtener valores dinámicamente.
        return datetime.now(timezone.utc)

Product.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='product_name', alias_priority=2),
 'price': FieldInfo(annotation=Union[float, NoneType], required=False, default=None),
 'is_available': FieldInfo(annotation=Union[bool, NoneType], required=False, default=None, alias='Esta disponible', alias_priority=2),
 'serial_number': FieldInfo(annotation=Union[int, NoneType], required=False, default=None),
 'creation_date': FieldInfo(annotation=str, required=False, default_factory=<lambda>)}

2. Crea una instancia de `Product` sin proporcionar la fecha de creación y verifica que se establezca correctamente.



In [131]:
datos = {
    "product_name": "Camiseta",
    "price": 29.99,
    "is_available": True,
    "extra_field": "Este campo será añadido"
}

pr1 = Product.model_validate(datos)
print(pr1.model_dump())


{'name': 'Camiseta', 'price': 29.99, 'is_available': True, 'serial_number': None, 'creation_date': datetime.datetime(2024, 10, 9, 11, 30, 8, 735229, tzinfo=datetime.timezone.utc), 'extra_field': 'Este campo será añadido'}


### Ejercicio 8: Serializadores customizados

1. Crea un serializador personalizado que convierta el nombre (`name`) a minúsculas al serializar.


In [134]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from datetime import datetime, timezone

class Product(BaseModel):
    model_config = ConfigDict(extra="allow")  # Ignorar campos adicionales

    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)
    serial_number: Optional[int] = None
    creation_date: str = Field(default_factory=lambda: Product.obtener_fecha_hora())

    @staticmethod
    def obtener_fecha_hora():
        # Esta función puede realizar cálculos u obtener valores dinámicamente.
        return datetime.now(timezone.utc)

    @field_serializer("name")
    def serialize_float(self, value):
        return value.lower()

Product.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='product_name', alias_priority=2),
 'price': FieldInfo(annotation=Union[float, NoneType], required=False, default=None),
 'is_available': FieldInfo(annotation=Union[bool, NoneType], required=False, default=None, alias='Esta disponible', alias_priority=2),
 'serial_number': FieldInfo(annotation=Union[int, NoneType], required=False, default=None),
 'creation_date': FieldInfo(annotation=str, required=False, default_factory=<lambda>)}

2. Utiliza este serializador al serializar una instancia de `Product` y verifica que el nombre esté en minúsculas.



In [136]:
datos = {
    "product_name": "Camiseta",
    "price": 29.99,
    "is_available": True,
    "extra_field": "Este campo será añadido"
}

pr1 = Product.model_validate(datos)
print(pr1.model_dump())

{'name': 'camiseta', 'price': 29.99, 'is_available': True, 'serial_number': None, 'creation_date': datetime.datetime(2024, 10, 9, 11, 58, 1, 546166, tzinfo=datetime.timezone.utc), 'extra_field': 'Este campo será añadido'}


### Ejercicio 9: Validadores customizados

1. Agrega un validador personalizado al campo `price` del modelo `Product` que asegure que el precio sea mayor de 0.


In [139]:
from pydantic import BaseModel, Field, ConfigDict
from typing import Optional, List
from datetime import datetime, timezone

class Product(BaseModel):
    model_config = ConfigDict(extra="allow")  # Ignorar campos adicionales

    name: str | None = Field(alias="product_name", default=None)
    price: Optional[float] = None
    is_available: Optional[bool] | None = Field(alias="Esta disponible", default=None)
    serial_number: Optional[int] = None
    creation_date: str = Field(default_factory=lambda: Product.obtener_fecha_hora())

    @staticmethod
    def obtener_fecha_hora():
        # Esta función puede realizar cálculos u obtener valores dinámicamente.
        return datetime.now(timezone.utc)

    @field_serializer("name")
    def serialize_float(self, value):
        return value.lower()

    @field_validator("price")
    @classmethod
    def validar_precio(cls, value):
        if value <= 0:
            raise ValueError("El precio debe ser mayor que 0")
        return value

Product.model_fields

{'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='product_name', alias_priority=2),
 'price': FieldInfo(annotation=Union[float, NoneType], required=False, default=None),
 'is_available': FieldInfo(annotation=Union[bool, NoneType], required=False, default=None, alias='Esta disponible', alias_priority=2),
 'serial_number': FieldInfo(annotation=Union[int, NoneType], required=False, default=None),
 'creation_date': FieldInfo(annotation=str, required=False, default_factory=<lambda>)}

2. Intenta crear una instancia de `Product` con un precio no válido y verifica el error de validación.

### Ejercicio 10: Modelos de datos anidados

1. Crea un modelo Pydantic llamado `Order` con campos como `product` (instancia de `Product`) y `quantity` (int).
2. Crea una instancia de `Order` con datos completos, incluyendo un producto y una cantidad.
